### Installation

In [5]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install jiwer
!pip install einops addict easydict

### Unsloth

Let's prepare the OCR model to our local first

In [1]:
from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

README-checkpoint.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

processor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'/content/deepseek_ocr'

In [6]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from transformers import AutoModel
import os
os.environ["UNSLOTH_WARN_UNINITIALIZED"] = '0'
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Qwen3-VL-8B-Instruct-bnb-4bit", # Qwen 3 vision support
    "unsloth/Qwen3-VL-8B-Thinking-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Instruct-bnb-4bit",
    "unsloth/Qwen3-VL-32B-Thinking-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "./deepseek_ocr",
    load_in_4bit = False, # Use 4bit to reduce memory use. False for 16bit LoRA.
    auto_model = AutoModel,
    trust_remote_code=True,
    unsloth_force_compile=True,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.4: Fast Deepseekocr patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at ./deepseek_ocr and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Mount drive and download dataset

In [9]:
from google.colab import drive
drive.mount("/content/drive")

import os, glob

# Thư mục bạn muốn có dataset
ROOT_DIR = "/content/drive/MyDrive/uit_hwdb_line"
DATA_DIR = os.path.join(ROOT_DIR, "UIT_HWDB_line")
ZIP_PATH = os.path.join(ROOT_DIR, "UIT_HWDB_line.zip")

os.makedirs(ROOT_DIR, exist_ok=True)

if os.path.isdir(DATA_DIR):
    print("✅ Đã có folder:", DATA_DIR)
else:
    print("📥 Chưa có dataset, bắt đầu tải về...")

    # Cài gdown nếu chưa có
    !pip install -q gdown

    # ID của file UIT-HWDB-line trên Google Drive (từ README GitHub)
    FILE_ID = "1Ul6r2lSP24PlZ6Biabhzg5Q2Wg8Qw6E-"  # UIT-HWDB-line

    # Tải file zip về Drive nếu chưa có
    if not os.path.exists(ZIP_PATH):
        !gdown --id "$FILE_ID" -O "$ZIP_PATH"
    else:
        print("📦 Đã có file zip:", ZIP_PATH)

    # Giải nén
    !unzip -q "$ZIP_PATH" -d "$ROOT_DIR"

    # Nếu cần, tìm thư mục vừa giải nén và in ra
    subdirs = [d for d in glob.glob(os.path.join(ROOT_DIR, "*")) if os.path.isdir(d)]
    print("Các thư mục con trong", ROOT_DIR, ":\n", "\n".join(subdirs))
    print("✅ Download + unzip xong. Kiểm tra xem trong đó có UIT_HWDB_line chưa.")


Mounted at /content/drive
✅ Đã có folder: /content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line


### Prepare train dataset

In [ ]:
from datasets import Dataset
from PIL import Image
import os, json

ROOT = "/content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line"
TRAIN_DIR = os.path.join(ROOT, "train_data")

print("TRAIN_DIR:", TRAIN_DIR)

# --- hàm đọc 1 batch: /.../train_data/250 (có label.json + các .jpg) ---
def load_batch(batch_dir):
    label_path = os.path.join(batch_dir, "label.json")
    if not os.path.isfile(label_path):
        print("Không thấy label.json trong", batch_dir)
        return []

    with open(label_path, "r", encoding="utf-8") as f:
        data = json.load(f)      # {"1.jpg": "text", ...}

    pairs = []
    for fname, txt in data.items():
        img_path = os.path.join(batch_dir, fname)
        if os.path.isfile(img_path):
            pairs.append({"image_path": img_path, "text": txt})
        else:
            print("Thiếu file ảnh:", img_path)
    return pairs

# --- đọc toàn bộ train_data ---
all_pairs = []
for name in sorted(os.listdir(TRAIN_DIR)):
    batch_dir = os.path.join(TRAIN_DIR, name)
    if not os.path.isdir(batch_dir):
        continue
    batch_pairs = load_batch(batch_dir)
    all_pairs.extend(batch_pairs)
    print(f"Batch {name}: {len(batch_pairs)} mẫu")

print("Tổng số mẫu train:", len(all_pairs))

TRAIN_DIR: /content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line/train_data
Batch 1: 16 mẫu
Batch 10: 1 mẫu
Batch 100: 14 mẫu
Batch 101: 36 mẫu
Batch 102: 14 mẫu
Batch 103: 40 mẫu
Batch 104: 32 mẫu
Batch 105: 40 mẫu
Batch 106: 31 mẫu
Batch 107: 19 mẫu
Batch 108: 21 mẫu
Batch 109: 20 mẫu
Batch 11: 14 mẫu
Batch 110: 44 mẫu
Batch 111: 27 mẫu
Batch 112: 30 mẫu
Batch 113: 21 mẫu
Batch 114: 35 mẫu
Batch 115: 31 mẫu
Batch 116: 29 mẫu
Batch 117: 48 mẫu
Batch 118: 59 mẫu
Batch 119: 29 mẫu
Batch 12: 32 mẫu
Batch 120: 64 mẫu
Batch 121: 28 mẫu
Batch 122: 46 mẫu
Batch 123: 16 mẫu
Batch 124: 29 mẫu
Batch 125: 29 mẫu
Batch 126: 30 mẫu
Batch 127: 30 mẫu
Batch 128: 14 mẫu
Batch 129: 19 mẫu
Batch 13: 10 mẫu
Batch 130: 45 mẫu
Batch 131: 10 mẫu
Batch 132: 19 mẫu
Batch 133: 30 mẫu
Batch 134: 21 mẫu
Batch 135: 28 mẫu
Batch 136: 25 mẫu
Batch 137: 24 mẫu
Batch 138: 20 mẫu
Batch 139: 27 mẫu
Batch 14: 34 mẫu
Batch 140: 9 mẫu
Batch 141: 75 mẫu
Batch 142: 47 mẫu
Batch 143: 11 mẫu
Batch 144: 11 mẫu
Batch 145: 47 mẫu


## Preprocess train dataset

In [ ]:
# --- tạo HF Dataset thô (chỉ path + text) ---
train_ds = Dataset.from_list(all_pairs)
print("Ví dụ 1 mẫu raw:", train_ds[0])

# --- chuyển về format 'messages' mà Unsloth DeepSeekOCRDataCollator yêu cầu ---
instruction = "<image>\nFree OCR. "

def to_conversation(example):
    try:
        img = Image.open(example["image_path"]).convert("RGB")
    except Exception as e:
        print("Lỗi mở ảnh:", example["image_path"], e)
        return {"messages": None}

    return {
        "messages": [
            {
                "role": "<|User|>",
                "content": instruction,
                "images": [img],            # ảnh dạng PIL.Image.Image
            },
            {
                "role": "<|Assistant|>",
                "content": example["text"],
            },
        ]
    }

train_conv = train_ds.map(
    to_conversation,
    remove_columns=train_ds.column_names,   # chỉ giữ lại "messages"
)

# bỏ các mẫu lỗi ảnh
train_conv = train_conv.filter(lambda ex: ex["messages"] is not None)


print("Số mẫu train_conv:", len(train_conv))
print("Ví dụ messages:")
print(train_conv[0]["messages"])
print("Kiểu ảnh:", type(train_conv[0]["messages"][0]["images"][0]))

Ví dụ 1 mẫu raw: {'image_path': '/content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line/train_data/1/1.jpg', 'text': 'KHÁI QUÁT VỀ BIỂN ĐẢO VIỆT NAM'}


Map:   0%|          | 0/7028 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7028 [00:00<?, ? examples/s]

Số mẫu train_conv: 7028
Ví dụ messages:
[{'content': '<image>\nFree OCR. ', 'images': [{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x08\x00\x00\x00\x00\xe3\x08\x02\x00\x00\x00\x9e\xfac,\x00\x01\x00\x00IDATx\x9c\x8c\xbdy\xd4nGU\xe7\xbf\xeb\x9cw\xbc7\x97\x10\x12\x12\x02\x99\x15B \xa3\x06\x82\x1d\x14"\x82d\x81\x80\xb6\xd2hk+\x86\xe5\x12\xed\x86%"\xa6\x01\x07hF\x81\x08(\xdd-`\xcb`k7\x82\x8c*\xda\t,\x10B&\x93@\x86\x85\x01\x12\x122\x87\x90\xe4\xe6\xde\xfb\x8e\xa7\xea\xf7\xc7\xee\xfa\xf2\xa9]\xe7\xcd\xaf\xcf\x1f\xefz\xde\xe79\xa7j\xd7\xde\xdf=\xd6p\xd2\xde\xbd{\xcdlqqqccc\x18\x0633\xb3\x94\x92\x99\x95R\xccl{{{qq\xb1\x942\x0cC)ekk+\xa5\x94R\x1a\xc7\xb1\x94RJI)M\xd3\xb4\xb0\xb0\x90s\xf6\xfb\x17\x16\x16\xbc\x91\x9c\xb37\xe5\xff\xfa7\xfe\xef\xc2\xc2\xc24M\xdeEJ\xc9\x9b\xd2\x9d\xe38noo;\r\xeah\x18\x06op\x9a\xa6\xc5\xc5E3\xcb9;\x01~\x95R\xfc\x1bo\xdf\x7f*\xa5xG\xc30\xa8#\xa7\xd6\xbf\x9c\xa6\xc9\x876\x8e\xa3\x99\xf9X\xfc\x91\x94\x92\xdf`f\xc30\xf8m9\xe7\x9c\xf38\x8e>\x1c\xb5\xc3\x91:\xcd~

## Save preprocessed train dataset to drive (run first time)

In [ ]:
from datasets import Dataset

SAVE_ROOT = "/content/drive/MyDrive/uit_hwdb_line/hf_datasets"

os.makedirs(SAVE_ROOT, exist_ok=True)

train_save_path = os.path.join(SAVE_ROOT, "train_conv")

train_conv.save_to_disk(train_save_path)

print("Đã lưu train_conv_train vào:", train_save_path)

Saving the dataset (0/5 shards):   0%|          | 0/7028 [00:00<?, ? examples/s]

Đã lưu train_conv_train vào: /content/drive/MyDrive/uit_hwdb_line/hf_datasets/train_conv


## Load preprocessed train dataset from drive

In [10]:
from datasets import load_from_disk
import os

SAVE_ROOT = "/content/drive/MyDrive/uit_hwdb_line/hf_datasets"

train_save_path = os.path.join(SAVE_ROOT, "train_conv")

train_conv = load_from_disk(train_save_path)

print("Loaded train:", len(train_conv))
print("Ví dụ 1 mẫu train:", train_conv[0]["messages"])

Loaded train: 7028
Ví dụ 1 mẫu train: [{'content': '<image>\nFree OCR. ', 'images': [{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x08\x00\x00\x00\x00\xe3\x08\x02\x00\x00\x00\x9e\xfac,\x00\x01\x00\x00IDATx\x9c\x8c\xbdy\xd4nGU\xe7\xbf\xeb\x9cw\xbc7\x97\x10\x12\x12\x02\x99\x15B \xa3\x06\x82\x1d\x14"\x82d\x81\x80\xb6\xd2hk+\x86\xe5\x12\xed\x86%"\xa6\x01\x07hF\x81\x08(\xdd-`\xcb`k7\x82\x8c*\xda\t,\x10B&\x93@\x86\x85\x01\x12\x122\x87\x90\xe4\xe6\xde\xfb\x8e\xa7\xea\xf7\xc7\xee\xfa\xf2\xa9]\xe7\xcd\xaf\xcf\x1f\xefz\xde\xe79\xa7j\xd7\xde\xdf=\xd6p\xd2\xde\xbd{\xcdlqqqccc\x18\x0633\xb3\x94\x92\x99\x95R\xccl{{{qq\xb1\x942\x0cC)ekk+\xa5\x94R\x1a\xc7\xb1\x94RJI)M\xd3\xb4\xb0\xb0\x90s\xf6\xfb\x17\x16\x16\xbc\x91\x9c\xb37\xe5\xff\xfa7\xfe\xef\xc2\xc2\xc24M\xdeEJ\xc9\x9b\xd2\x9d\xe38noo;\r\xeah\x18\x06op\x9a\xa6\xc5\xc5E3\xcb9;\x01~\x95R\xfc\x1bo\xdf\x7f*\xa5xG\xc30\xa8#\xa7\xd6\xbf\x9c\xa6\xc9\x876\x8e\xa3\x99\xf9X\xfc\x91\x94\x92\xdf`f\xc30\xf8m9\xe7\x9c\xf38\x8e>\x1c\xb5\xc3\x91:\xcd~\x

## Split train dataset to train and val dataset

In [11]:
from datasets import DatasetDict
split_ratio = 0.1

dataset_dict = train_conv.train_test_split(
    test_size = split_ratio,
    shuffle   = True,
    seed      = 42,
)

train_conv_train = dataset_dict["train"]
val_conv         = dataset_dict["test"]

print("Số mẫu train:", len(train_conv_train))
print("Số mẫu valid:", len(val_conv))
print("Ví dụ 1 mẫu valid:", val_conv[0]["messages"])

Số mẫu train: 6325
Số mẫu valid: 703
Ví dụ 1 mẫu valid: [{'content': '<image>\nFree OCR. ', 'images': [{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x08\x00\x00\x00\x00\x83\x08\x02\x00\x00\x00\x96\x0elv\x00\x01\x00\x00IDATx\x9c\x8c\xbdy\xb4eGu\xdf\xbf\xeb\x9c\xfb\x86\xee\xd7\xdd\x92Z\x12\x1a0\x1a\x10FBFBf\x90\x01\x19!\x90\x8dq\x0c\x18l \x98\x98\x84\x05\xb1\xb1qVp\xbc\x92,{\xad\xfc\x95?\xe2\xb5b\xe7\x17g\xf0\x10\xece\x07\xc2\x10\x83-\xe4\t#3H\x02$$\xd0\x80\x84\x001\x08\x83F\xa4\x96Zju\xf7\x1b\xef\xa9\xfa\xfd\xb1]\xdf|j\xd7}J\xce\x1fo\xddw\xef9U\xbb\xf6\xf0\xddC\r\'=\xf8\xe0\x83\xa5\x94\x94R)\xa5\x94bf\xc30\xf8\xe7\x94RJ\xc9\xea\x15\xeeI)\xe5\x9c\xfd{3\xf3\xfbs\xce\xa5\x94a\x18\xc6q\xd4\xcd\xfa\xe0\r\x96z\xf9=9\xe7\xd0\x11\xef\x0c]\xe83\xef\xf7\x16\xc6q4\xb3i\x9a\x9c\x80i\x9a\xfc\x1b\xffU\x83r:\x87a\x98\xcf\xe7\xb3\xd9L7\xabq\xefq\x1cG\xff\xde\xbfYZZ\xda\xd9\xd9)\xa5,//onn\x8a\x06\xe7\x03i#\xc1\xba\x86a\xf0\xef\xbdA\xbf\x7f6\x9b\xf9\xf7\xa4\xd0o\x98\xcdf\xd34\xe9\x1b\xa7\xc7\xc

## Preprocess test dataset

In [ ]:
from datasets import Dataset
import os, json

ROOT = "/content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line"
TEST_DIR = os.path.join(ROOT, "test_data")

def load_batch(batch_dir):
    label_path = os.path.join(batch_dir, "label.json")
    if not os.path.isfile(label_path):
        print("Không thấy label.json trong", batch_dir)
        return []
    with open(label_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    pairs = []
    for fname, txt in data.items():
        img_path = os.path.join(batch_dir, fname)
        if os.path.isfile(img_path):
            pairs.append({"image_path": img_path, "text": txt})
    return pairs

test_pairs = []
for name in sorted(os.listdir(TEST_DIR)):
    batch_dir = os.path.join(TEST_DIR, name)
    if not os.path.isdir(batch_dir):
        continue
    test_pairs.extend(load_batch(batch_dir))

test_ds = Dataset.from_list(test_pairs)
print("Số mẫu test:", len(test_ds))
print("Ví dụ:", test_ds[0])

Số mẫu test: 201
Ví dụ: {'image_path': '/content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line/test_data/250/1.jpg', 'text': 'Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với'}


## Save prepocessed test dataset to drive

In [ ]:
test_pairs = []
for name in sorted(os.listdir(TEST_DIR)):
    batch_dir = os.path.join(TEST_DIR, name)
    if not os.path.isdir(batch_dir):
        continue
    test_pairs.extend(load_batch(batch_dir))

test_ds = Dataset.from_list(test_pairs)
print("Số mẫu test:", len(test_ds))
print("Ví dụ:", test_ds[0])

SAVE_ROOT = "/content/drive/MyDrive/uit_hwdb_line/hf_datasets"
os.makedirs(SAVE_ROOT, exist_ok=True)

test_save_path = os.path.join(SAVE_ROOT, "test_ds")
test_ds.save_to_disk(test_save_path)
print("Đã lưu test_ds vào:", test_save_path)

Số mẫu test: 201
Ví dụ: {'image_path': '/content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line/test_data/250/1.jpg', 'text': 'Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với'}


Saving the dataset (0/1 shards):   0%|          | 0/201 [00:00<?, ? examples/s]

Đã lưu test_ds vào: /content/drive/MyDrive/uit_hwdb_line/hf_datasets/test_ds


## Load preprocessed test dataset from drive

In [12]:
from datasets import load_from_disk
test_ds = load_from_disk("/content/drive/MyDrive/uit_hwdb_line/hf_datasets/test_ds")
print(len(test_ds), test_ds[0])

201 {'image_path': '/content/drive/MyDrive/uit_hwdb_line/UIT_HWDB_line/test_data/250/1.jpg', 'text': 'Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với'}


## Inference baseline model with test dataset

In [13]:
from unsloth import FastVisionModel

FastVisionModel.for_inference(model)

DeepseekOCRForCausalLM(
  (model): DeepseekOCRModel(
    (embed_tokens): Embedding(129280, 1280)
    (layers): ModuleList(
      (0): DeepseekV2DecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=1280, out_features=1280, bias=False)
          (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
          (v_proj): Linear(in_features=1280, out_features=1280, bias=False)
          (o_proj): Linear(in_features=1280, out_features=1280, bias=False)
        )
        (mlp): DeepseekV2MLP(
          (gate_proj): Linear(in_features=1280, out_features=6848, bias=False)
          (up_proj): Linear(in_features=1280, out_features=6848, bias=False)
          (down_proj): Linear(in_features=6848, out_features=1280, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): DeepseekV2RMSNorm((1280,), eps=1e-06)
        (post_attention_layernorm): DeepseekV2RMSNorm((1280,), eps=1e-06)
      )
      (1-11): 11 x DeepseekV2DecoderLayer(

In [14]:
from jiwer import cer
from tqdm.auto import tqdm

PROMPT = "<image>\nFree OCR. "

def ocr_one(model, tokenizer, image_path):
    res = model.infer(
        tokenizer,
        prompt        = PROMPT,
        image_file    = image_path,
        output_path   = ".",      # không quan trọng nếu không lưu file
        base_size     = 1024,
        image_size    = 640,
        crop_mode     = True,
        save_results  = False,    # không cần file, ta lấy từ res
        test_compress = False,
        eval_mode     = True,     # 🔴 QUAN TRỌNG: để infer trả về text
    )

    # res có thể là string hoặc dict tuỳ version
    if isinstance(res, str):
        return res.strip()
    if isinstance(res, dict) and "text" in res:
        return res["text"].strip()
    return str(res).strip()


ex = test_ds[0]
pred_text = ocr_one(model, tokenizer, ex["image_path"])

print("GT  :", ex["text"])
print("Pred:", pred_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
GT  : Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với
Pred: Thử thường, Bộ tài nguyên và môi trường Đảng Hùng Võ đãn đạo về:


In [15]:
import os, json

def eval_and_save(model, tokenizer, test_ds, tag="baseline", max_samples=201):
    """
    tag: 'baseline' hoặc 'finetuned'
    max_samples: số mẫu test muốn chạy (None = full)
    """
    save_root = "/content/drive/MyDrive/uit_hwdb_line/results"
    os.makedirs(save_root, exist_ok=True)
    out_path = os.path.join(save_root, f"results_{tag}.json")

    n = len(test_ds) if max_samples is None else min(len(test_ds), max_samples)

    all_samples = []
    scores = []

    for i in tqdm(range(n), desc=f"Evaluating {tag}"):
        ex = test_ds[i]
        img_path = ex["image_path"]
        gt = ex["text"]

        try:
            pred = ocr_one(model, tokenizer, img_path)
        except Exception as e:
            print(f"Lỗi ở mẫu {i}: {e}")
            continue

        sample_cer = cer(gt, pred)
        scores.append(sample_cer)

        all_samples.append({
            "id": i,
            "image_path": img_path,
            "ground_truth": gt,
            "prediction": pred,
            "cer": sample_cer,
        })

    if not scores:
        print("Không tính được CER mẫu nào.")
        return None

    avg_cer = sum(scores) / len(scores)

    # Gói vào dict lớn
    result = {
        "tag": tag,
        "num_samples": len(all_samples),
        "average_cer": avg_cer,
        "samples": all_samples,
    }

    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    print(f"✅ {tag} CER trung bình trên {len(all_samples)} mẫu = {avg_cer:.4f}")
    print("Đã lưu kết quả vào:", out_path)
    return result


In [ ]:
from unsloth import FastVisionModel

FastVisionModel.for_inference(model)  # đảm bảo model đang ở mode infer

baseline_results = eval_and_save(
    model,
    tokenizer,
    test_ds,
    tag="baseline",
    max_samples=201,  # hoặc None nếu bạn muốn chạy full
)

Evaluating baseline:   0%|          | 0/201 [00:00<?, ?it/s]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE: 

<h3>Baseline Model Performance: 32.68% Mean Character Error Rate (CER) for this sample!</h3>

# Let's finetune Deepseek-OCR !

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

**[NEW]** We also support finetuning ONLY the vision part of the model, or ONLY the language part. Or you can select both! You can also select to finetune the attention or the MLP layers!

In [16]:
model = FastVisionModel.get_peft_model(
    model,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

Unsloth: Making `model.base_model.model.model` require gradients


In [17]:
# @title Create datacollator

import torch
import math
from dataclasses import dataclass
from typing import Dict, List, Any, Tuple
from PIL import Image, ImageOps
from torch.nn.utils.rnn import pad_sequence
import io

from deepseek_ocr.modeling_deepseekocr import (
    format_messages,
    text_encode,
    BasicImageTransform,
    dynamic_preprocess,
)

@dataclass
class DeepSeekOCRDataCollator:
    """
    Args:
        tokenizer: Tokenizer
        model: Model
        image_size: Size for image patches (default: 640)
        base_size: Size for global view (default: 1024)
        crop_mode: Whether to use dynamic cropping for large images
        train_on_responses_only: If True, only train on assistant responses (mask user prompts)
    """
    tokenizer: Any
    model: Any
    image_size: int = 640
    base_size: int = 1024
    crop_mode: bool = True
    image_token_id: int = 128815
    train_on_responses_only: bool = True

    def __init__(
        self,
        tokenizer,
        model,
        image_size: int = 640,
        base_size: int = 1024,
        crop_mode: bool = True,
        train_on_responses_only: bool = True,
    ):
        self.tokenizer = tokenizer
        self.model = model
        self.image_size = image_size
        self.base_size = base_size
        self.crop_mode = crop_mode
        self.image_token_id = 128815
        self.dtype = model.dtype  # Get dtype from model
        self.train_on_responses_only = train_on_responses_only

        self.image_transform = BasicImageTransform(
            mean=(0.5, 0.5, 0.5),
            std=(0.5, 0.5, 0.5),
            normalize=True
        )
        self.patch_size = 16
        self.downsample_ratio = 4

        # Get BOS token ID from tokenizer
        if hasattr(tokenizer, 'bos_token_id') and tokenizer.bos_token_id is not None:
            self.bos_id = tokenizer.bos_token_id
        else:
            self.bos_id = 0
            print(f"Warning: tokenizer has no bos_token_id, using default: {self.bos_id}")

    def deserialize_image(self, image_data) -> Image.Image:
        """Convert image data (bytes dict or PIL Image) to PIL Image in RGB mode"""
        if isinstance(image_data, Image.Image):
            return image_data.convert("RGB")
        elif isinstance(image_data, dict) and 'bytes' in image_data:
            image_bytes = image_data['bytes']
            image = Image.open(io.BytesIO(image_bytes))
            return image.convert("RGB")
        else:
            raise ValueError(f"Unsupported image format: {type(image_data)}")

    def calculate_image_token_count(self, image: Image.Image, crop_ratio: Tuple[int, int]) -> int:
        """Calculate the number of tokens this image will generate"""
        num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
        num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

        width_crop_num, height_crop_num = crop_ratio

        if self.crop_mode:
            img_tokens = num_queries_base * num_queries_base + 1
            if width_crop_num > 1 or height_crop_num > 1:
                img_tokens += (num_queries * width_crop_num + 1) * (num_queries * height_crop_num)
        else:
            img_tokens = num_queries * num_queries + 1

        return img_tokens

    def process_image(self, image: Image.Image) -> Tuple[List, List, List, List, Tuple[int, int]]:
        """
        Process a single image based on crop_mode and size thresholds

        Returns:
            Tuple of (images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio)
        """
        images_list = []
        images_crop_list = []
        images_spatial_crop = []

        if self.crop_mode:
            # Determine crop ratio based on image size
            if image.size[0] <= 640 and image.size[1] <= 640:
                crop_ratio = (1, 1)
                images_crop_raw = []
            else:
                images_crop_raw, crop_ratio = dynamic_preprocess(
                    image, min_num=2, max_num=9,
                    image_size=self.image_size, use_thumbnail=False
                )

            # Process global view with padding
            global_view = ImageOps.pad(
                image, (self.base_size, self.base_size),
                color=tuple(int(x * 255) for x in self.image_transform.mean)
            )
            images_list.append(self.image_transform(global_view).to(self.dtype))

            width_crop_num, height_crop_num = crop_ratio
            images_spatial_crop.append([width_crop_num, height_crop_num])

            # Process local views (crops) if applicable
            if width_crop_num > 1 or height_crop_num > 1:
                for crop_img in images_crop_raw:
                    images_crop_list.append(
                        self.image_transform(crop_img).to(self.dtype)
                    )

            # Calculate image tokens
            num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
            num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

            tokenized_image = ([self.image_token_id] * num_queries_base + [self.image_token_id]) * num_queries_base
            tokenized_image += [self.image_token_id]

            if width_crop_num > 1 or height_crop_num > 1:
                tokenized_image += ([self.image_token_id] * (num_queries * width_crop_num) + [self.image_token_id]) * (
                    num_queries * height_crop_num)

        else:  # crop_mode = False
            crop_ratio = (1, 1)
            images_spatial_crop.append([1, 1])

            # For smaller base sizes, resize; for larger, pad
            if self.base_size <= 640:
                resized_image = image.resize((self.base_size, self.base_size), Image.LANCZOS)
                images_list.append(self.image_transform(resized_image).to(self.dtype))
            else:
                global_view = ImageOps.pad(
                    image, (self.base_size, self.base_size),
                    color=tuple(int(x * 255) for x in self.image_transform.mean)
                )
                images_list.append(self.image_transform(global_view).to(self.dtype))

            num_queries = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)
            tokenized_image = ([self.image_token_id] * num_queries + [self.image_token_id]) * num_queries
            tokenized_image += [self.image_token_id]

        return images_list, images_crop_list, images_spatial_crop, tokenized_image, crop_ratio

    def process_single_sample(self, messages: List[Dict]) -> Dict[str, Any]:
            """
            Process a single conversation into model inputs.
            """

            # --- 1. Setup ---
            images = []
            for message in messages:
                if "images" in message and message["images"]:
                    for img_data in message["images"]:
                        if img_data is not None:
                            pil_image = self.deserialize_image(img_data)
                            images.append(pil_image)

            if not images:
                raise ValueError("No images found in sample. Please ensure all samples contain images.")

            tokenized_str = []
            images_seq_mask = []
            images_list, images_crop_list, images_spatial_crop = [], [], []

            prompt_token_count = -1 # Index to start training
            assistant_started = False
            image_idx = 0

            # Add BOS token at the very beginning
            tokenized_str.append(self.bos_id)
            images_seq_mask.append(False)

            for message in messages:
                role = message["role"]
                content = message["content"]

                # Check if this is the assistant's turn
                if role == "<|Assistant|>":
                    if not assistant_started:
                        # This is the split point. All tokens added *so far*
                        # are part of the prompt.
                        prompt_token_count = len(tokenized_str)
                        assistant_started = True

                    # Append the EOS token string to the *end* of assistant content
                    content = f"{content.strip()} {self.tokenizer.eos_token}"

                # Split this message's content by the image token
                text_splits = content.split('<image>')

                for i, text_sep in enumerate(text_splits):
                    # Tokenize the text part
                    tokenized_sep = text_encode(self.tokenizer, text_sep, bos=False, eos=False)
                    tokenized_str.extend(tokenized_sep)
                    images_seq_mask.extend([False] * len(tokenized_sep))

                    # If this text is followed by an <image> tag
                    if i < len(text_splits) - 1:
                        if image_idx >= len(images):
                            raise ValueError(
                                f"Data mismatch: Found '<image>' token but no corresponding image."
                            )

                        # Process the image
                        image = images[image_idx]
                        img_list, crop_list, spatial_crop, tok_img, _ = self.process_image(image)

                        images_list.extend(img_list)
                        images_crop_list.extend(crop_list)
                        images_spatial_crop.extend(spatial_crop)

                        # Add image placeholder tokens
                        tokenized_str.extend(tok_img)
                        images_seq_mask.extend([True] * len(tok_img))

                        image_idx += 1 # Move to the next image

            # --- 3. Validation and Final Prep ---
            if image_idx != len(images):
                raise ValueError(
                    f"Data mismatch: Found {len(images)} images but only {image_idx} '<image>' tokens were used."
                )

            # If we never found an assistant message, we're in a weird state
            # (e.g., user-only prompt). We mask everything.
            if not assistant_started:
                print("Warning: No assistant message found in sample. Masking all tokens.")
                prompt_token_count = len(tokenized_str)

            # Prepare image tensors
            images_ori = torch.stack(images_list, dim=0)
            images_spatial_crop_tensor = torch.tensor(images_spatial_crop, dtype=torch.long)

            if images_crop_list:
                images_crop = torch.stack(images_crop_list, dim=0)
            else:
                images_crop = torch.zeros((1, 3, self.base_size, self.base_size), dtype=self.dtype)

            return {
                "input_ids": torch.tensor(tokenized_str, dtype=torch.long),
                "images_seq_mask": torch.tensor(images_seq_mask, dtype=torch.bool),
                "images_ori": images_ori,
                "images_crop": images_crop,
                "images_spatial_crop": images_spatial_crop_tensor,
                "prompt_token_count": prompt_token_count, # This is now accurate
            }

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        """Collate batch of samples"""
        batch_data = []

        # Process each sample
        for feature in features:
            try:
                processed = self.process_single_sample(feature['messages'])
                batch_data.append(processed)
            except Exception as e:
                print(f"Error processing sample: {e}")
                continue

        if not batch_data:
            raise ValueError("No valid samples in batch")

        # Extract lists
        input_ids_list = [item['input_ids'] for item in batch_data]
        images_seq_mask_list = [item['images_seq_mask'] for item in batch_data]
        prompt_token_counts = [item['prompt_token_count'] for item in batch_data]

        # Pad sequences
        input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        images_seq_mask = pad_sequence(images_seq_mask_list, batch_first=True, padding_value=False)

        # Create labels
        labels = input_ids.clone()

        # Mask padding tokens
        labels[labels == self.tokenizer.pad_token_id] = -100

        # Mask image tokens (model shouldn't predict these)
        labels[images_seq_mask] = -100

        # Mask user prompt tokens when train_on_responses_only=True (only train on assistant responses)
        if self.train_on_responses_only:
            for idx, prompt_count in enumerate(prompt_token_counts):
                if prompt_count > 0:
                    labels[idx, :prompt_count] = -100

        # Create attention mask
        attention_mask = (input_ids != self.tokenizer.pad_token_id).long()

        # Prepare images batch (list of tuples)
        images_batch = []
        for item in batch_data:
            images_batch.append((item['images_crop'], item['images_ori']))

        # Stack spatial crop info
        images_spatial_crop = torch.cat([item['images_spatial_crop'] for item in batch_data], dim=0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "images": images_batch,
            "images_seq_mask": images_seq_mask,
            "images_spatial_crop": images_spatial_crop,
        }

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

We use our new `DeepSeekOCRDataCollator` which will help in our vision finetuning setup.

In [18]:
from unsloth import FastVisionModel

# Đảm bảo đang ở mode train (bật grad, checkpointing, v.v.)
FastVisionModel.for_training(model)

# Collator đã được định nghĩa sẵn trong notebook Unsloth
data_collator = DeepSeekOCRDataCollator(
    tokenizer  = tokenizer,
    model      = model,
    image_size = 640,
    base_size  = 1024,
    crop_mode  = True,
    train_on_responses_only = True,  # chỉ tính loss trên câu trả lời
)


In [19]:
from transformers import Trainer, TrainingArguments
from unsloth import is_bf16_supported
FastVisionModel.for_training(model) # Enable for training!
data_collator = DeepSeekOCRDataCollator(
    tokenizer=tokenizer,
    model = model,
    image_size=640,
    base_size=1024,
    crop_mode=True,
    train_on_responses_only=True,
)
trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = data_collator, # Must use!
    train_dataset = train_conv_train,
    eval_dataset = val_conv,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 85,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 1e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        fp16 = not is_bf16_supported(),  # Use fp16 if bf16 is not supported
        bf16 = is_bf16_supported(),  # Use bf16 if supported
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases
        dataloader_num_workers=2,
        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
    ),
)


/tmp/ipython-input-2926612885.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.273 GB of memory reserved.


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,325 | Num Epochs = 1 | Total steps = 85
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 77,509,632 of 3,413,615,872 (2.27% trained)
Unsloth: Not an error, but DeepseekOCRForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
Unsloth: Will smartly offload gradients to save VRAM!
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])


Step,Training Loss
1,1.640800
2,2.178600
3,1.756100
4,1.425200
5,1.877900
6,1.448500
7,1.308700
8,1.415300
9,1.887800
10,1.113700


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([7, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE: 

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


In [22]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2294.2876 seconds used for training.
38.24 minutes used for training.
Peak reserved memory = 9.699 GB.
Peak reserved memory for training = 2.426 GB.
Peak reserved memory % of max memory = 65.796 %.
Peak reserved memory for training % of max memory = 16.457 %.


In [23]:
import os, json

RESULT_ROOT = "/content/drive/MyDrive/uit_hwdb_line/results"
os.makedirs(RESULT_ROOT, exist_ok=True)

log_path = os.path.join(RESULT_ROOT, "finetune_log_history_v2.json")

# trainer.state.log_history là list các dict:
# [{"loss": ..., "learning_rate": ..., "epoch": ..., "step": ...}, {"eval_loss": ...}, ...]
log_history = trainer.state.log_history

with open(log_path, "w", encoding="utf-8") as f:
    json.dump(log_history, f, ensure_ascii=False, indent=2)

print("✅ Đã lưu log fine-tune vào:", log_path)
print("Tổng số entry log:", len(log_history))

✅ Đã lưu log fine-tune vào: /content/drive/MyDrive/uit_hwdb_line/results/finetune_log_history_v2.json
Tổng số entry log: 86


In [24]:
from unsloth import FastVisionModel

FastVisionModel.for_inference(model)   # đưa model về mode infer

save_dir = "/content/drive/MyDrive/uit_hwdb_line/deepseek_ocr_uit_merged_v2"

model.save_pretrained_merged(
    save_dir,
    tokenizer,
    save_method = "merged_16bit",   # hoặc "merged_4bit" tuỳ notebook hướng dẫn
)

print("Đã lưu model merged vào:", save_dir)

Detected local model directory: /content/deepseek_ocr
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [02:21<00:00, 141.21s/it]


Copied model-00001-of-000001.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [03:59<00:00, 239.05s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/uit_hwdb_line/deepseek_ocr_uit_merged_v2`
Đã lưu model merged vào: /content/drive/MyDrive/uit_hwdb_line/deepseek_ocr_uit_merged_v2


<a name="Inference"></a>
### Inference
Let's run the model!

In [25]:
ex = test_ds[0]
pred_text = ocr_one(model, tokenizer, ex["image_path"])

print("GT  :", ex["text"])
print("Pred:", pred_text)

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
GT  : Thứ trưởng Bộ Tài nguyên & môi trường Đặng Hùng Võ tâm đắc với
Pred: Thủ trưởng Bộ tài nguyên và môi trường Đảng Hưng Võ đã nói rõ rệt:


In [26]:
finetuned_results = eval_and_save(
    model,
    tokenizer,
    test_ds,
    tag="finetuned_v2",
    max_samples=201,
)


Evaluating finetuned_v2:   0%|          | 0/201 [00:00<?, ?it/s]

BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([9, 100, 1280])
BASE: 